# setup

In [ ]:
run_test=True
DO_TRAIN_VOLS = True
DO_TRAIN_PRCS = True
Provisional=True
MODE = 'tuning'

# run_test=False
# DO_TRAIN_VOLS = False   # 是否运行 VOLS 的 VOL_GRID 训练
# DO_TRAIN_PRCS = False  # 是否运行 PRCS 的 LMSE x VOL_GRID 交叉训练
Provisional=False
MODE = 'prod'

'''
Provisional run_test 	 	MODE 	 	VOLS/PRCS 	Tables itrs
TRUE 	 	TRUE 	 	tuning 	完整网格→切片前2 	 前2个表 0
TRUE 	 	TRUE 	 	prod 	 	BEST配置 	 	前2个表 	 	0
TRUE 	 	FALSE 	tuning 	完整网格→切片前2 	 全部 	 	0
TRUE 	 	FALSE 	prod 	 	BEST配置 	 	全部 	 	range(itr)
FALSE 	 TRUE 	 	tuning 	完整网格 	 	全部 	 0
FALSE 	 TRUE 	 	prod 	 	BEST配置 	 	全部 	 	range(itr)
FALSE 	 FALSE 	 tuning 	完整网格 	 	全部 	 0
FALSE 	 FALSE 	 prod 	 	BEST配置 	 	全部 	 	range(itr)
'''

epochs=(1,1,1,0,12) if Provisional else (15,500,3,0,12)

cutdate='2025-11-18'
cutdateend='2026-11-18'
cutdate='2024-12-18'
cutdateend='2025-04-30'

valData=''
use_amp=True
use_amp=False
# valData='2512'
testData=''
testData='new/'
initial_lr = .01

weight=(0.81,1.01,0.03,0)
itr=3
metrics_cnt=6
import torch,sys
# Provisional=Provisional and run_test

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  mydrive='/content/drive/MyDrive/volrt/'
  checkpoints=mydrive+"checkpoints/"
  source_path=mydrive+"git/Crossformer"
  tables = [ 'volvTSLA.csv', 'volvAAPL.csv'] if (Provisional and run_test) else [ 'volvTSLA.csv', 'volvAAPL.csv', 'volvGOOG.csv','volvNVDA.csv', 'volvMETA.csv', 'volvMSFT.csv', 'volvAMZN.csv', ]
  batch_size=int(torch.cuda.get_device_properties(0).total_memory/1e9+.5)*25
  import os,sys
  if not os.path.exists(source_path):
    %cd $mydrive/git
    !git clone https://github.com/jerronl/Crossformer.git
    %cd $source_path
  else:
    %cd $source_path
    # !git reset --hard HEAD
    # !git pull origin master
  %pip install einops #scikit-learn
  sys.path.append( source_path)
else:
  tables = [ 'volvAAPLt.csv']
  use_amp=True
  testData=''
  mydrive= 'E:/mydoc/git/trade/analyics/'
  checkpoints="G:/git/Crossformer1/checkpoints/"
  batch_size=32
  sys.path.append("G:/git/Crossformer1/")
  # itr=1
  %load_ext autoreload
  %autoreload 2
%cd $mydrive


E:\mydoc\git\trade\analyics


d:\ProgramData\miniconda3\envs\s1\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Funcs

In [ ]:
import os
import re
import traceback
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
import torch

from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS, data_columns, data_names
from data.data_def import set_cat
from utils.tools import init_args, update_args
from utils.metrics import metric_names

set_cat(20)
args = init_args()
data_parser = {}

COMMON_SETTINGS = {
    "e_layers": 5,
    "d_model": 512,
    "lradj": "type2",
    "checkpoints": checkpoints,
}


def generate_configs(grid, mode_idx):
    active_keys = []
    values_lists = []
    for key, (vals, in_vols, in_prcs) in grid.items():
        is_active = in_vols if mode_idx == 1 else in_prcs
        if is_active:
            active_keys.append(key)
            values_lists.append(vals)
    return [dict(zip(active_keys, c)) for c in itertools.product(*values_lists)]


def apply_config_override(parser_config: dict, ds: str, config_dict: dict) -> dict:
    temp_config = (
        parser_config.copy() if ds not in parser_config else parser_config[ds].copy()
    )
    temp_config.update(config_dict)

    if "weight_over" in config_dict and "over_weight" not in temp_config:
        temp_config["over_weight"] = config_dict["weight_over"]

    if ds not in parser_config:
        return temp_config
    parser_config[ds] = temp_config
    return parser_config


def build_parser(mode, ds, extra_query="", root_override=None, config_defaults=None):
    root = root_override or mydrive + (valData if mode == "insample" else testData)
    q = ""
    if mode == "oos":
        q = f"date>'#{cutdate}' and date<'#{cutdateend}'"
        if extra_query:
            q += " and " + extra_query
    elif mode == "insample":
        q = extra_query

    idx_w = 0 if ds == "vols" else 1
    idx_ow = 2 if ds == "vols" else 3

    defaults = config_defaults or {}

    ds_config = {
        **COMMON_SETTINGS,
        "root_path": root,
        "data_path": tables,
        "data": ds,
        "weight": defaults.get("weight", weight[idx_w]),
        "over_weight": defaults.get("weight_over", weight[idx_ow]),
        **defaults,
    }

    if q:
        ds_config["query"] = re.sub(r"^\s*and\s+", "", q)

    parser = {} if ds == "vols" else {"vols": {"checkpoints": checkpoints}}
    parser[ds] = ds_config
    return parser


def build_train_parser(root_path=None):
    base = {
        "patience": epochs[0],
        "train_epochs": TRAIN_EPOCHS,
        "learning_rate": initial_lr,
        "data_split": [0.7, 0.15, 0.15],
        "batch_size": batch_size * 2 // 5,
        "e_layers": 5,
        "d_model": 512,
        "lradj": "type2",
        "checkpoints": checkpoints,
        "root_path": root_path or mydrive + valData,
        "data_path": tables,
        "profile_mode": False,
        "use_amp": use_amp,
    }
    return {
        "vols": {**base, "data": "vols", "weight": weight[0], "over_weight": weight[2]},
        "prcs": {**base, "data": "prcs", "weight": weight[1], "over_weight": weight[3]},
    }


def get_feature_names(ds_name, in_len):
    cols = data_columns(ds_name)
    return data_names(cols, 1)[0]


def get_trained_epochs(exp, setting_str, ds, idx):
    trained_ep = exp.get_epochs()
    if trained_ep != -1:
        return trained_ep

    folder_path = os.path.join(checkpoints, setting_str) + ds + str(idx)
    file_path = os.path.join(folder_path, "checkpoint.pth")
    if os.path.exists(file_path):
        try:
            state = torch.load(file_path, weights_only=False)
            if isinstance(state, dict) and "epoch" in state:
                return state["epoch"]
        except Exception:
            return "Err"
    return "Not Fnd"


def prepare_preds_trues_for_plot(preds, trues, ycat=0):
    preds, trues = np.asarray(preds), np.asarray(trues)
    if ycat > 0:
        reg_dim = preds.shape[1] - ycat
        min_dim = min(trues.shape[1], reg_dim)
        return preds[:, :min_dim], trues[:, :min_dim]
    return preds, trues


def _metrics_text(t, p):
    diff = p - t
    metrics = {
        "mae": np.mean(np.abs(diff)),
        "rmse": np.sqrt(np.mean(diff**2)),
        "mape": np.mean(np.abs(diff) / (np.abs(t) + 1e-12)),
    }
    return f"mae={metrics['mae']:.4g}\nrmse={metrics['rmse']:.4g}\nmape={metrics['mape']:.3g}"


def _metrics_text_prcs_ic_iv(t_iv, p_iv, t_ic, p_ic_label):
    diff = p_iv - t_iv
    mae = np.mean(np.abs(diff))
    rmse = np.sqrt(np.mean(diff**2))
    mape = np.mean(np.abs(diff) / (np.abs(t_iv) + 1e-12))

    accr = np.nan
    valid_mask = ~np.isnan(p_ic_label) & ~np.isnan(t_ic)
    if len(t_ic) > 0 and (np.unique(t_ic).size <= 20) and (~np.isnan(t_ic)).sum() > 0:
        if np.unique(p_ic_label).size <= 20:
            accr = np.mean(p_ic_label.round()[valid_mask] == t_ic.round()[valid_mask])

    base_text = f"mae(IV)={mae:.4g}\nrmse(IV)={rmse:.4g}"
    return (
        f"accr(IC)={accr:.3f}\n{base_text}"
        if not np.isnan(accr)
        else f"{base_text}\nmape(IV)={mape:.3g}"
    )


def _compute_limits_like_old_regplot(trues_list, preds_list, feat_idx, feat_name):
    left, right = np.inf, -np.inf
    for trues, preds in zip(trues_list, preds_list):
        t, p = trues[:, feat_idx], preds[:, feat_idx]
        mask = ~np.isnan(t) & ~np.isnan(p)
        if not mask.any():
            continue
        t, p = t[mask], p[mask]

        l_curr, r_curr = (
            (np.min(t), np.max(t))
            if feat_name[:3] in ["dtm", "pmc"]
            else (max(np.min(t), -5), min(np.max(t), 5))
        )
        p_min, p_max = (
            (np.min(p), np.max(p))
            if feat_name[:3] in ["dtm", "pmc"]
            else (max(np.min(p), -5), min(np.max(p), 5))
        )

        left = min(left, l_curr, p_min)
        right = max(right, r_curr, p_max)

    if not np.isfinite(left) or not np.isfinite(right) or left == right:
        return -1, 1
    return left, right


def plot_residual_heatmap_family(
    preds_raw,
    trues_raw,
    feature_names,
    title,
    bins=80,
    ycat=0,
    cols=4,
    figsize_per_cell=4,
):
    preds, trues = prepare_preds_trues_for_plot(preds_raw, trues_raw, ycat=ycat)
    effective_dim = min(trues.shape[1], len(feature_names))
    rows = int(np.ceil(effective_dim / cols))

    fig, axes = plt.subplots(
        rows,
        cols,
        figsize=(figsize_per_cell * cols, figsize_per_cell * rows),
        squeeze=False,
        constrained_layout=True,
    )

    for i in range(effective_dim):
        ax = axes.flatten()[i]
        t, p = trues[:, i], preds[:, i]
        mask = ~np.isnan(t) & ~np.isnan(p)
        t, p = t[mask], p[mask]

        if len(t) == 0:
            ax.set_title(feature_names[i])
            ax.axis("off")
            continue

        metrics_text = (
            _metrics_text_prcs_ic_iv(t, p, t, preds[mask, 4])
            if (ycat > 0 and i == 0)
            else _metrics_text(t, p)
        )
        left, right = _compute_limits_like_old_regplot(
            [trues_raw], [preds_raw], i, feature_names[i]
        )

        ax.hist2d(t, p, bins=bins, range=[[left, right], [left, right]], norm="log")
        ax.plot([left, right], [left, right], "w--", alpha=0.45)
        ax.set_xlim(left, right)
        ax.set_ylim(left, right)
        ax.set_xlabel("True")
        ax.set_ylabel("Pred")
        ax.set_title(feature_names[i], fontsize=12)
        ax.text(
            0.02,
            0.98,
            metrics_text,
            transform=ax.transAxes,
            va="top",
            ha="left",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.25", fc="black", ec="none", alpha=0.35),
            color="white",
            zorder=3,
        )

    for i in range(effective_dim, rows * cols):
        axes.flatten()[i].axis("off")
    if title:
        fig.suptitle(title, fontsize=14)
    plt.show()
    plt.close(fig)


def _get_plot_config(all_preds_raw, feature_names, cols):
    if not all_preds_raw or not all_preds_raw[0].shape[1]:
        return None
    effective_dim = min(all_preds_raw[0].shape[1], len(feature_names))
    if len(feature_names) < effective_dim:
        feature_names += [f"F{i}" for i in range(len(feature_names), effective_dim)]

    return {
        "effective_dim": effective_dim,
        "feature_names": feature_names[:effective_dim],
        "rows": (effective_dim + cols - 1) // cols,
        "colors_errors": plt.rcParams["axes.prop_cycle"]
        .by_key()
        .get("color", [f"C{i}" for i in range(10)]),
        "colors_residuals": [
            "#1f77b4",
            "#ff7f0e",
            "#2ca02c",
            "#d62728",
            "#9467bd",
            "#8c564b",
            "#e377c2",
            "#7f7f7f",
            "#bcbd22",
            "#17becf",
        ],  # truncated for brevity
    }


def plot_multi_stock_errors(
    all_preds_raw,
    all_trues_raw,
    stock_labels,
    feature_names,
    title="",
    cols=4,
    figsize_per_cell=4,
    config=None,
):
    config = config or _get_plot_config(all_preds_raw, feature_names, cols)
    if not config:
        return

    rows = config["rows"]
    top_margin = 0.905 + rows * 0.005
    if not title:
        top_margin += 0.02
    fig = plt.figure(figsize=(cols * figsize_per_cell, rows * figsize_per_cell))
    outer = gridspec.GridSpec(rows, cols, wspace=0.25, hspace=0.4)

    for i in range(config["effective_dim"]):
        ax = fig.add_subplot(outer[divmod(i, cols)])
        t_all, e_all = [], []

        for j, (p_reg, t_reg) in enumerate(zip(all_preds_raw, all_trues_raw)):
            t, p = t_reg[:, i], p_reg[:, i]
            mask = ~np.isnan(t) & ~np.isnan(p)
            if not mask.any():
                continue
            t, p = t[mask], p[mask]
            e = p - t
            t_all.append(t)
            e_all.append(e)
            ax.scatter(
                t,
                e,
                s=4,
                alpha=0.4,
                color=config["colors_errors"][j % len(config["colors_errors"])],
                label=stock_labels[j] if i == 0 else None,
                zorder=2,
            )

        if not t_all:
            ax.set_title(config["feature_names"][i])
            ax.axis("off")
            continue

        t_all, e_all = np.concatenate(t_all), np.concatenate(e_all)
        ax.axhline(0.0, color="k", linestyle="--", linewidth=0.8, alpha=0.7)

        if np.unique(t_all).size > 1:
            m, c = np.polyfit(t_all, e_all, 1)
            ax.plot(
                [t_all.min(), t_all.max()],
                m * np.array([t_all.min(), t_all.max()]) + c,
                "-.",
                lw=1.0,
                color="tab:orange",
            )

        mae = float(np.mean(np.abs(e_all)))
        rmse = float(np.sqrt(np.mean(e_all**2)))
        over_mask = e_all > 0
        under_mask = e_all < 0
        over_frac = float(np.mean(over_mask)) if over_mask.size else 0.0
        over_mae = float(np.mean(np.abs(e_all[over_mask]))) if over_mask.any() else 0.0
        under_mae = (
            float(np.mean(np.abs(e_all[under_mask]))) if under_mask.any() else 0.0
        )

        metrics = (
            f"RMSE: {rmse:.4g}\n"
            f"MAE: {mae:.4g}\n"
            f"over%: {over_frac*100:4.1f}\n"
            f"MAE_over: {over_mae:.4g}\n"
            f"MAE_under: {under_mae:.4g}"
        )
        ax.text(
            0.03,
            0.97,
            metrics,
            transform=ax.transAxes,
            fontsize=8,
            va="top",
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
        )
        ax.set_title(config["feature_names"][i], fontsize=10)
        ax.set_xlabel("true")
        ax.set_ylabel("error = pred - true")
        ax.grid(True, linestyle="--", alpha=0.4)
        if i == 0:
            ax.legend(fontsize=7, loc="lower right")

    if title:
        fig.suptitle(title, fontsize=14, y=0.99)
    fig.subplots_adjust(
        top=top_margin,
        hspace=0.35,
        wspace=0.25,
    )
    plt.show()


def plot_multi_stock_residuals(
    all_preds_raw,
    all_trues_raw,
    stock_labels,
    feature_names,
    ycat=0,
    title="",
    bins=80,
    cols=4,
    figsize_per_cell=4,
):
    config = _get_plot_config(all_preds_raw, feature_names, cols)
    if not config:
        return

    rows = config["rows"]
    top_margin = 0.905 + rows * 0.005
    if not title:
        top_margin += 0.02
    fig = plt.figure(figsize=(cols * figsize_per_cell, rows * figsize_per_cell))
    outer = gridspec.GridSpec(rows, cols, wspace=0.25, hspace=0.4)

    for i in range(config["effective_dim"]):
        ax = fig.add_subplot(outer[divmod(i, cols)])
        ax_hist = ax.twinx()
        left, right = 999, -999
        max_hist_density = 0
        legend_handles = []

        for j, (p_reg, t_reg) in enumerate(zip(all_preds_raw, all_trues_raw)):
            t, p = t_reg[:, i], p_reg[:, i]
            mask = ~np.isnan(t) & ~np.isnan(p)
            if not mask.any():
                continue
            t, p = t[mask], p[mask]

            color = config["colors_residuals"][j % len(config["colors_residuals"])]
            try:
                h_vals, _, _ = ax_hist.hist(
                    t, bins=bins, density=True, alpha=0.15, color=color, zorder=1
                )
                if len(h_vals) > 0:
                    max_hist_density = max(max_hist_density, np.max(h_vals))
            except:
                pass

            l_curr, r_curr = (
                (np.min(t), np.max(t))
                if feature_names[i][:3] in ["dtm", "pmc"]
                else (max(np.min(t), -5), min(np.max(t), 5))
            )
            left, right = min(left, l_curr), max(right, r_curr)
            ax.scatter(t, p, s=0.5, alpha=0.4, color=color, zorder=2)

            try:
                m, c = np.polyfit(t, p, 1)
                (line,) = ax.plot(
                    [t.min(), t.max()],
                    m * np.array([t.min(), t.max()]) + c,
                    "--",
                    color=color,
                    lw=1.5,
                    alpha=0.4,
                    label=stock_labels[j],
                )
                legend_handles.append(line)
            except:
                pass

        cur_xlim = ax.get_xlim()
        cur_ylim = ax.get_ylim()
        if left == 999:
            left = cur_xlim[0]
        if right == -999:
            right = cur_xlim[1]
        final_min = min(-0.01, min(left, cur_xlim[0], cur_ylim[0]))
        final_max = max(0.01, max(right, cur_xlim[1], cur_ylim[1]))
        ax.set_xlim(final_min, final_max)
        ax.set_ylim(final_min, final_max)
        ax_hist.set_ylim(0, max_hist_density * 3 if max_hist_density > 0 else 1)
        ax_hist.axis("off")
        ax.set_title(config["feature_names"][i], y=1.01, fontsize=11)
        ax.legend(handles=legend_handles, fontsize=8, loc="best")

        all_t = np.concatenate([t[:, i] for t in all_trues_raw if t.shape[1] > i])
        all_p = np.concatenate([p[:, i] for p in all_preds_raw if p.shape[1] > i])
        mask = ~np.isnan(all_t) & ~np.isnan(all_p)
        if mask.any():
            rmse = np.sqrt(np.mean((all_t[mask] - all_p[mask]) ** 2))
            mae = np.mean(np.abs(all_t[mask] - all_p[mask]))
            metrics = f"RMSE: {rmse:.4f}\nMAE: {mae:.4f}"
            ax.text(
                0.05,
                0.95,
                metrics,
                transform=ax.transAxes,
                fontsize=9,
                va="top",
                bbox=dict(boxstyle="round", fc="white", alpha=0.85),
            )

    if title:
        fig.suptitle(title, fontsize=14, y=0.99)
    fig.subplots_adjust(
        top=top_margin,
        hspace=0.35,
        wspace=0.25,
    )
    plt.show()

    if all_preds_raw:
        plot_multi_stock_errors(
            all_preds_raw,
            all_trues_raw,
            stock_labels,
            feature_names,
            title=f"{title} - Prediction Errors",
            cols=cols,
            figsize_per_cell=figsize_per_cell,
            config=config,
        )


def plot_prcs_distribution_detailed(
    results_list,
    model_idx,
    ycat,
    dep_var,
    tables_list,
    cols=4,
    title_prefix="PRCS Distribution",
):
    if not results_list or ycat == 0:
        return

    n_categories = ycat
    x_ticks = np.arange(1, n_categories + 1)
    outer_rows = (len(results_list) - 1) // cols + 1
    fig = plt.figure(figsize=(16, 4 * outer_rows))
    outer = gridspec.GridSpec(outer_rows, cols, wspace=0.3, hspace=0.45)
    ls = ["-", "--", ":"]

    print(f"\n--- Plotting {title_prefix} for Model m{model_idx} ---")
    for j, (preds_raw, trues_raw, _) in enumerate(results_list):
        probs, labels = np.asarray(preds_raw)[:, -ycat:], np.round(
            np.asarray(trues_raw)[:, 0]
        ).astype(int)
        mask = (~np.isnan(labels)) & (labels >= 1) & (labels <= ycat)
        if not mask.any():
            continue

        probs, labels = probs[mask], labels[mask]
        if probs.size > 0 and (probs.max() > 1.05 or probs.min() < -0.05):
            exp_probs = np.exp(probs - np.max(probs, axis=1, keepdims=True))
            probs = exp_probs / (np.sum(exp_probs, axis=1, keepdims=True) + 1e-12)

        df = pd.DataFrame(probs, columns=[str(i) for i in range(1, ycat + 1)]).assign(
            True_Label=labels
        )
        summed = df.groupby("True_Label").mean().T

        sub_gs = outer[j // cols, j % cols].subgridspec(2, 1, hspace=0.0)
        valid_cols = sorted([c for c in summed.columns if 1 <= c <= ycat])
        for k in (0, 1):
            ax = fig.add_subplot(sub_gs[k, 0])
            ax.hist(
                labels,
                bins=np.arange(1, ycat + 2) - 0.5,
                alpha=0.2,
                density=True,
                color="gray",
                label="True",
                zorder=1,
            )

            ci = 0
            for m in range(k, len(valid_cols), 2):
                col = valid_cols[m]
                ax.plot(
                    [int(x) for x in summed.index],
                    summed[col],
                    label=f"cat {col}",
                    linestyle=ls[col // (ycat // 3 + 1) % 3],
                    color=f"C{ci % 10}",
                    alpha=0.7,
                    zorder=2,
                )
                ax.text(
                    col,
                    summed[col][str(col)],
                    f"{col}",
                    fontsize=8,
                    va="bottom",
                    ha="center",
                    color=f"C{ci % 10}",
                    zorder=3,
                )
                ci += 1

            ax.set_ylim(0, 0.2)
            ax.set_xlim(0.5, ycat + 1.5)
            ax.set_xticks(x_ticks)
            ax.set_xticklabels([str(i) for i in x_ticks], rotation=-45, fontsize=8)
            ax.grid(axis="y", linestyle="--", alpha=0.3)

            if k == 0:
                ax.set_title(tables_list[j], fontsize=10)
                plt.setp(ax.get_xticklabels(), visible=False)
                ax.set_xlabel("")
            else:
                ax.set_xlabel("Predicted Class Index", fontsize=9)

            ax.legend(
                loc="center left",
                bbox_to_anchor=(1.05, 0.5),
                fontsize=7,
                framealpha=0.6,
            )

    fig.suptitle(f"{title_prefix} - Model m{model_idx}", fontsize=14)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()
    plt.close()


def generate_final_summary(summary_records):
    if not summary_records:
        return
    df = pd.DataFrame(summary_records)
    meta = [c for c in ["dataset", "idx", "horizon", "epochs"] if c in df]
    metrics = sorted(
        [c for c in df.columns if c in set(metric_names[0] + metric_names[1])]
    )
    params = sorted([c for c in df.columns if c not in meta + metrics])
    print(
        df[meta + params + metrics].to_string(
            float_format=lambda x: f"{x:.4g}" if isinstance(x, (float, int)) else str(x)
        )
    )


def _init_exp_and_setting(parser_config: dict, ds: str, itr_idx: int = 0):
    args = init_args()
    return (
        args,
        update_args(args, parser_config, itr=itr_idx, arg_set=ds),
        Exp_crossformer(args),
    )


def run_test_one_setting(
    exp,
    setting_str,
    ds,
    dep_var,
    ycat,
    title_prefix,
    by_stock=False,
    stock_tables=None,
    plot_individual_heatmaps=True,
    test_data=None,
    itr_idx=0,
):
    m_list, all_p, all_t = [], [], []
    ds_tag = ds + str(itr_idx)

    def _plot(p, t, title):
        if plot_individual_heatmaps:
            print(f"\n--- Residual Heatmap: {title} ---")
            plot_residual_heatmap_family(p, t, dep_var, title=title, ycat=ycat)

    if by_stock:
        for tb in stock_tables:
            p, t, m = exp.test(setting_str, ds_tag, True, data_path=[tb], inverse=True)
            m_list.append(m)
            all_p.append(p)
            all_t.append(t)
            _plot(p, t, f"{title_prefix} {tb}")
        return m_list, all_p, all_t

    p, t, m = exp.test(setting_str, ds_tag, True, inverse=True, test_data=test_data)
    _plot(p, t, title_prefix)
    return [m], ([p] if test_data else [p]), ([t] if test_data else [t])


def train_unified_all():
    global args, UNIFIED_CONFIGS, active_grid_indices, DO_TRAIN_VOLS, DO_TRAIN_PRCS
    base_parser = build_train_parser()

    print(
        f"\n=== Unified Training (Mode: {MODE}, Active: {len(active_grid_indices)}) ==="
    )
    for idx in active_grid_indices:
        if idx >= len(UNIFIED_CONFIGS):
            break
        cfg = UNIFIED_CONFIGS[idx].copy()
        ds, itr_idx = cfg.pop("target_ds"), cfg.pop("itr_idx", 0)

        if (ds == "vols" and not DO_TRAIN_VOLS) or (ds == "prcs" and not DO_TRAIN_PRCS):
            continue

        DatasetMTS.clear()
        curr_parser = {k: v.copy() for k, v in base_parser.items()}
        curr_parser[ds] = apply_config_override(curr_parser[ds], ds, cfg)

        # Training Cycles
        start_lr = curr_parser[ds]["learning_rate"]
        for cycle in range(epochs[2]):
            curr_parser[ds]["learning_rate"] = start_lr / (3.0**cycle)
            args, setting, exp = _init_exp_and_setting(curr_parser, ds, itr_idx)

            print(
                f"\n[Idx {idx}|Itr {itr_idx}|{ds.upper()}|Cyc {cycle+1}] LR={curr_parser[ds]['learning_rate']:.6f} | {setting}"
            )
            try:
                exp.train(setting, ds + str(itr_idx))
                if MODE == "prod":
                    exp.test(
                        setting,
                        ds + str(itr_idx),
                        True,
                        data_path=[tables[-1]],
                        inverse=True,
                    )
            except Exception as e:
                print(f"--- FAILED Idx {idx} --- {e}")
                traceback.print_exc()


def test_unified(
    mode: str, target_ds: str, by_stock=True, plot_residuals=True, plot_stock=False
):
    global args, UNIFIED_CONFIGS, active_grid_indices, tables
    summary = []

    indices = [
        i
        for i in active_grid_indices
        if i < len(UNIFIED_CONFIGS) and UNIFIED_CONFIGS[i]["target_ds"] == target_ds
    ]
    if not indices:
        return print(f"No configs for {target_ds}")

    print(f"\n=== Testing {target_ds.upper()} ({mode}) - {len(indices)} configs ===")
    for idx in indices:
        cfg = UNIFIED_CONFIGS[idx].copy()
        ds, itr_idx = cfg.pop("target_ds"), cfg.pop("itr_idx", 0)
        ycat = data_columns(ds).get("ycat", 0) if ds == "prcs" else 0

        base_parser = build_parser(mode, ds)
        base_parser[ds] = apply_config_override(base_parser[ds], ds, cfg)

        args, setting, exp = _init_exp_and_setting(base_parser, ds, itr_idx)
        trained_ep = get_trained_epochs(exp, setting, ds, itr_idx)

        # if trained_ep == "Not Fnd": print(f"Skipping {setting}, model not found"); continue

        try:
            dep_var = get_feature_names(ds, getattr(args, "seq_len", 1))
            title = f"{ds.upper()} {mode} [Idx {idx}|Itr {itr_idx}]"

            m_list, p_fam, t_fam = run_test_one_setting(
                exp,
                setting,
                ds,
                dep_var,
                ycat,
                title,
                by_stock,
                tables,
                plot_stock,
                itr_idx=itr_idx,
            )

            if by_stock and p_fam and plot_residuals:
                print(f"\n--- AGGREGATED: {title} ---")
                plot_multi_stock_residuals(
                    p_fam, t_fam, tables, dep_var, ycat, title=f"AGG {title}"
                )
                if ds == "prcs" and ycat > 0:
                    plot_prcs_distribution_detailed(
                        list(zip(p_fam, t_fam, m_list)),
                        idx,
                        ycat,
                        dep_var,
                        tables,
                        title_prefix=f"AGG {title}",
                    )

            summary.append(
                {
                    "dataset": ds,
                    "idx": idx,
                    "itr": itr_idx,
                    "epochs": trained_ep,
                    **cfg,
                    **dict(
                        zip(metric_names[1 if ycat > 0 else 0], np.mean(m_list, axis=0))
                    ),
                }
            )

        except Exception as e:
            print(f"--- FAILED Test Idx {idx} --- {e}")
            traceback.print_exc()

    generate_final_summary(summary)
    if len(summary) > 1:
        print("\n--- Metrics Summary ---")
        plot_metric_summary(summary)
    return summary


def test_vols_per_horizon(mode="insample", plot_residuals=True, best_idx=-1):
    global args, UNIFIED_CONFIGS, tables
    summary, ds, horizons = [], "vols", [1, 2, 3, 4, 5]

    indices = [
        i
        for i in active_grid_indices
        if i < len(UNIFIED_CONFIGS) and UNIFIED_CONFIGS[i]["target_ds"] == ds
    ]
    if best_idx != -1 and best_idx in range(len(UNIFIED_CONFIGS)):
        indices = [best_idx]
    if not indices:
        return

    print(f"\n--- VOLS Horizon Test ({len(indices)} configs) ---")
    for idx in indices:
        cfg = UNIFIED_CONFIGS[idx].copy()
        ds, itr_idx = cfg.pop("target_ds"), cfg.pop("itr_idx", 0)

        base_parser = build_parser(mode, ds)
        parser_load = base_parser.copy()
        parser_load[ds] = apply_config_override(parser_load[ds], ds, cfg)

        args, setting, exp = _init_exp_and_setting(parser_load, ds, itr_idx)
        trained_ep = get_trained_epochs(exp, setting, ds, itr_idx)
        # if trained_ep == "Not Fnd": continue

        dep_var, orig_q = (
            get_feature_names(ds, getattr(args, "seq_len", 1)),
            exp.args.query,
        )
        h_preds, h_trues, h_names = [], [], []

        for h in horizons:
            exp.args.query = build_parser(mode, ds, extra_query=f"floor(horizon)=={h}")[
                ds
            ].get("query", "")
            try:
                dataset, loader = exp._get_data(flag="test", data=ds)
                m_list, p, t = run_test_one_setting(
                    exp,
                    setting,
                    ds,
                    dep_var,
                    0,
                    "",
                    False,
                    None,
                    False,
                    (dataset, loader),
                    itr_idx,
                )

                if m_list:
                    summary.append(
                        {
                            "dataset": ds,
                            "idx": idx,
                            "itr": itr_idx,
                            "horizon": h,
                            "epochs": trained_ep,
                            **cfg,
                            **dict(zip(metric_names[0], m_list[0])),
                        }
                    )
                    h_preds.append(p[0])
                    h_trues.append(t[0])
                    h_names.append(f"H {h}")
            except:
                pass

        exp.args.query = orig_q
        if plot_residuals and h_preds:
            plot_multi_stock_residuals(
                h_preds,
                h_trues,
                h_names,
                dep_var,
                0,
                title=f"VOLS Horizons [Idx {idx}]",
            )

    generate_final_summary(summary)
    plot_metric_summary(summary)
    return summary


def plot_metric_summary(records):
    if not records:
        return
    df = pd.DataFrame(records)
    metrics = [m for m in metric_names[0] + metric_names[1] if m in df.columns]

    rows = (len(metrics) + 1) // 2
    fig, axes = plt.subplots(rows, 2, figsize=(15, 5 * rows))
    for i, m in enumerate(metrics):
        ax = axes.flatten()[i]
        if "horizon" in df:
            for idx in df["idx"].unique():
                sub = df[df["idx"] == idx].sort_values("horizon")
                ax.plot(sub["horizon"], sub[m], "o-", label=f"Cfg {idx}")
            ax.legend()
        else:
            ax.plot(df["idx"], df[m], "o-")
        ax.set_title(m.upper())
        ax.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()
    plt.close()


# Params

In [3]:
PARAM_GRID_VOLS = {
    "lambda_var":  ([0.00050, 0.00055, 0.00060], True, True),
    "gain_reg":    ([1.5, 2.0], True, True),
    "weight_over": ([1.4, 1.5, 1.6], True, True),

    # 固定的最优值，不再扫描
    "lambda_tail": ([0.0], False, True),
    "lambda_anti": ([0.1], False, True),
    "dist_alpha":  ([1.0], False, True),
    "lambda_dist": ([0.0], False, True),
}

PARAM_GRID_PRCS = {
    # 围绕 0.0004 做轻微上下微调
    "lambda_var":  ([0.00035, 0.00040, 0.00045], False, True),

    # gain_reg 只看 0.8 / 1.0 / 1.2，都是“正常有梯度”的范围
    "gain_reg":    ([0.8, 1.0, 1.2],             False, True),

    # weight_over：基准 1.2，再看看 1.0 会不会稍微放松一点
    "weight_over": ([1.0, 1.2],                  False, True),

    # 这一轮先严格完全兼容老版：这些都锁死为 0
    "lambda_tail": ([0.0],                       False, True),
    "lambda_anti": ([0.0],                       False, True),
    "lambda_dist": ([0.0],                       False, True),

    # 老 baseline 用的是分类 + 回归等权
    "lambda_mse":  ([1.0],                       False, True),

    # 先固定为 1.0（和之前老跑法一致），下一轮再单独扫 dist_alpha
    "dist_alpha":  ([1.0],                       False, True),
}

vols_configs_raw = []
prcs_configs_raw = [
    {"dist_alpha": 1.0, "gain_reg": 1.12, "lambda_var": 0.0004, "weight_over": 1.0},
    {"dist_alpha": 1.0, "gain_reg": 1.18, "lambda_var": 0.0004, "weight_over": 1.0},
    {"dist_alpha": 1.0, "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 0.95},
    {"dist_alpha": 1.0, "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.05},
    {"dist_alpha": 1.1, "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.0},
    {"dist_alpha": 1.1, "gain_reg": 1.12, "lambda_var": 0.0004, "weight_over": 0.95},
    {"dist_alpha": 1.1, "gain_reg": 1.12, "lambda_var": 0.0004, "weight_over": 1.05},
    {"dist_alpha": 1.1, "gain_reg": 1.18, "lambda_var": 0.0004, "weight_over": 1.0},
    # A Group
    {"dist_alpha": 1.15, "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.0},
    {"dist_alpha": 1.2,  "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.0},
    # B Group
    {"dist_alpha": 1.0, "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.02},
    {"dist_alpha": 1.0, "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.08},
    # C Group
    {"dist_alpha": 1.1,  "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.02},
    {"dist_alpha": 1.15, "gain_reg": 1.15, "lambda_var": 0.0004, "weight_over": 1.08},
    # EMD Group
    {"dist_alpha": 1.0, "gain_reg": 1.12, "lambda_emd": 0.001, "weight_over": 1.0},
    {"dist_alpha": 1.0, "gain_reg": 1.18, "lambda_emd": 0.001, "weight_over": 1.0},
    {"dist_alpha": 1.0, "gain_reg": 1.15, "lambda_emd": 0.001, "weight_over": 0.95},
    {"dist_alpha": 1.0, "gain_reg": 1.15, "lambda_emd": 0.001, "weight_over": 1.05},
    {"dist_alpha": 1.1, "gain_reg": 1.15, "lambda_emd": 0.001, "weight_over": 1.0},
]


BEST_VOLS = {"lambda_var": 0.0005, "gain_reg": 1.5, "weight_over": 1.4, "lambda_anti": 0.1, "dist_alpha": 1.0}
BEST_PRCS = {"dist_alpha": 1.1, "gain_reg": 1.18, "lambda_mse": 1.0, "lambda_var": 0.0004, "weight_over": 1.0}


In [4]:

if MODE == "prod":
    vols_configs_raw = [BEST_VOLS] if DO_TRAIN_VOLS else []
    prcs_configs_raw = [BEST_PRCS] if DO_TRAIN_PRCS else []
else:
    if DO_TRAIN_VOLS and not vols_configs_raw: vols_configs_raw = generate_configs(PARAM_GRID_VOLS, 1)
    if DO_TRAIN_PRCS and not prcs_configs_raw: prcs_configs_raw = generate_configs(PARAM_GRID_PRCS, 2)


train_itr_list = [0] if (MODE == 'tuning' or (Provisional and run_test)) else list(range(itr))
defaults_map = {
    "vols": {"weight": weight[0], "weight_over": weight[2]},
    "prcs": {"weight": weight[1], "weight_over": weight[3]}
}

loop_itrs = train_itr_list

UNIFIED_CONFIGS = []
for ds, configs in [("vols", vols_configs_raw), ("prcs", prcs_configs_raw)]:
    defaults = defaults_map[ds]
    for cfg in configs:
        full_cfg = {**defaults, **cfg} 
        
        
        for itr_idx in loop_itrs:
            UNIFIED_CONFIGS.append({**full_cfg, "target_ds": ds, "itr_idx": itr_idx})

# Provisional Filter
if Provisional and MODE == "tuning":
    v_cnt = len(vols_configs_raw) * len(loop_itrs)
    p_cnt = len(prcs_configs_raw) * len(loop_itrs)
    UNIFIED_CONFIGS = (UNIFIED_CONFIGS[:min(2, v_cnt)] if v_cnt else []) + \
                      (UNIFIED_CONFIGS[v_cnt:v_cnt+min(2, p_cnt)] if p_cnt else [])

# Active Range Calculation
TRAIN_EPOCHS = min(30, epochs[1]) if MODE == "tuning" else epochs[1]
total = len(UNIFIED_CONFIGS)
start, end = epochs[3], min(epochs[3] + epochs[4], total)
active_grid_indices = [0] + list(range(start, end)) if (epochs[0] < 2 and start > 0) else list(range(start, end))

print(f"\n{'='*80}\nConfig Summary: Total={total}, Active={len(active_grid_indices)}, Epochs={TRAIN_EPOCHS}")
print(f"  Mode={MODE}, Test={run_test}, Provisional={Provisional}")
print(f"{'='*80}")
for i in active_grid_indices:
    if i < total:
        c = UNIFIED_CONFIGS[i]
        p_str = ", ".join([f"{k}={v:.4g}" if isinstance(v, float) else f"{k}={v}" for k, v in c.items() if k not in ["target_ds", "itr_idx"]])
        print(f"  [{i}] {c['target_ds'].upper()}|itr{c['itr_idx']}: {p_str}")
print(f"{'='*80}\n")


Config Summary: Total=6, Active=6, Epochs=500
  Mode=prod, Test=True, Provisional=False
  [0] VOLS|itr0: weight=0.81, weight_over=1.4, lambda_var=0.0005, gain_reg=1.5, lambda_anti=0.1, dist_alpha=1
  [1] VOLS|itr1: weight=0.81, weight_over=1.4, lambda_var=0.0005, gain_reg=1.5, lambda_anti=0.1, dist_alpha=1
  [2] VOLS|itr2: weight=0.81, weight_over=1.4, lambda_var=0.0005, gain_reg=1.5, lambda_anti=0.1, dist_alpha=1
  [3] PRCS|itr0: weight=1.01, weight_over=1, dist_alpha=1.1, gain_reg=1.18, lambda_mse=1, lambda_var=0.0004
  [4] PRCS|itr1: weight=1.01, weight_over=1, dist_alpha=1.1, gain_reg=1.18, lambda_mse=1, lambda_var=0.0004
  [5] PRCS|itr2: weight=1.01, weight_over=1, dist_alpha=1.1, gain_reg=1.18, lambda_mse=1, lambda_var=0.0004



# train

In [ ]:
if run_test == False:
        train_unified_all()

In [ ]:
if not run_test:
    from google.colab import runtime
    runtime.unassign()

# test
##  PART A — In-sample (test split of same source)
Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:
if run_test:
            test_unified("insample", "vols")

## In-sample - Prcs Per-Stock

Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:
if run_test:

           test_unified("insample", "prcs")


## In-sample - Horizon


In [ ]:
if run_test:
        test_vols_per_horizon(mode="insample")

# PART B — Out-of-sample (testData, cutdate+)
## OOS - Vols Per-Stock

In [5]:
if run_test:
        test_unified("oos", "vols")


=== Testing VOLS (oos) - 3 configs ===
Use GPU: cuda:0
 suc to load G:/git/Crossformer1/checkpoints/Cf_vols_itr0_sw_dist_alpha1.0_gain_reg1.5_lambda_anti0.1_lambda_var0.0005_over_weight0.03_weight0.81_weight_over1.4vols0/crossformer.pkl 


KeyboardInterrupt: 

## hlc

In [ ]:
if run_test:
        test_unified("oos", "prcs")

## horizon


In [ ]:
if run_test:

        test_vols_per_horizon(mode="oos")

# end

In [ ]:
plt.show()
from google.colab import runtime
runtime.unassign()